# Projeto Big Data & Data Mining

## Consumo de propagandas: impacto dos anúncios na decisão de compra dos consumidores

### Imports

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from dotenv import load_dotenv
import sys
sys.path.append("../utils/funcoes")
from menor_e_maior_idade import imputar_age_max, imputar_age_min

### Armazenamento

In [ ]:
df = pd.read_csv("../data/Social_Media_Advertising.csv")
df.head()

### Análise

In [ ]:
df.shape

In [ ]:
print("Primeiras 5 linhas do DataFrame: ")
print(df.head(5))

In [ ]:
print("Informações Gerais do DataFrame: ")
df.info()

In [ ]:
df.describe()

### Tratamentos

#### Target_Audience

In [ ]:
# Engenharia de Atributos: 
# Classificação de público-alvo (target audience)
print(df['Target_Audience'].unique())
print("\n--------Frequencia----------")
df['Target_Audience'].value_counts()

In [ ]:
# Separação de gênero em novas colunas e extação de faixa etária
df['_Target_Audience'] = df['Target_Audience'].str.extract(r'^(Men|Women|All Ages)', expand=False)
df[['Age_Min', 'Age_Max']] = df['Target_Audience'].str.extract(r'(\d+)-(\d+)', expand=True)

In [ ]:
#Conversão das idades em núméricos

df['Age_Max'] = pd.to_numeric(df['Age_Max'], errors='coerce').astype('Int16') # conversão direta para inteiro
df['Age_Min'] = pd.to_numeric(df['Age_Min'], errors='coerce').astype('Int16')
print(df['Age_Max'].head(5))

In [ ]:
print(df['_Target_Audience'].value_counts())

Imputação de dados de All_ages

In [ ]:
#Busca de não registrados
lista_ids = df.loc[df["Age_Min"].isna(), "Campaign_ID"].tolist()
i = 0
while i < 5 and len(lista_ids):
 print(lista_ids[i])
 i+=1


In [ ]:
ids = [727501, 320183, 592659, 567173, 431414]
print(pd.concat([df[df["Campaign_ID"] == i] for i in ids]))



In [ ]:
import importlib
import menor_e_maior_idade
importlib.reload(menor_e_maior_idade)

# Mínimo
df['Age_Min'] = imputar_age_min(df['Age_Min'])
# Máximo
df['Age_Max'] = imputar_age_max(df['Age_Max'])

In [ ]:
print(df['Age_Min'].info(),
      df['Age_Max'].info(),)

In [ ]:
ids = [727501, 320183, 592659, 567173, 431414]
print(pd.concat([df[df["Campaign_ID"] == i] for i in ids]))


Limpeza

In [ ]:
df = df.drop(columns=["Target_Audience"])

#### Duração

In [ ]:
# Duração de object para int
df['Duration'].unique()

In [ ]:
df = df.rename(columns={'Duration': 'Duration_in_Days'})
print(df['Duration_in_Days'].head(10).info())

In [ ]:
df["Duration_in_Days"] = df["Duration_in_Days"].str.replace("Days", " ", regex=False)
df["Duration_in_Days"] = df["Duration_in_Days"].str.strip()

In [ ]:
print(df['Duration_in_Days'].head(5))

In [ ]:
df["Duration_in_Days"] = pd.to_numeric(df["Duration_in_Days"], errors="coerce").astype("Int64")

In [ ]:
print(df['Duration_in_Days'].head(5))

#### Data

In [ ]:
# Data para datetime
df['Date'].head(3)

In [ ]:
df['Date'].head(10).info()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d", errors="coerce")
df['Date'].head(10).info()

In [ ]:
print(df['Date'].head(3))

#### Custo_Aquisicao

In [ ]:
df['Acquisition_Cost'].unique()[:20]

In [ ]:
df['Acquisition_Cost'] = (
    df['Acquisition_Cost']
    .replace(r'[\$,]', '', regex=True)
    .astype(float)
)
display(df.head(4))

### Visualização e exportação para Preparo de modelos preditivos, armazenamento, plotagem e relatório

#### View info final

In [ ]:
df.info()

#### Exportação

In [ ]:
df.to_csv("../Data/df_Social_Media_Tratado.csv",index=False)

#### Armazenamento

In [ ]:
load_dotenv()

In [ ]:
user = os.getenv("PGUSER")
password = os.getenv("PGPASS")
host = os.getenv("PGHOST")
port = os.getenv("PGPORT")
database = os.getenv("PGDB")

In [ ]:
DATABASE_URL = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
print(DATABASE_URL)

In [ ]:
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("conexão função seleção ç ã é")
print("UTF-8 funcionando")


In [ ]:
# Configuração com o banco

from pathlib import Path

sql_path = Path("sql") / "database_config.sql"
print("Executando script SQL database_config")

In [ ]:

try:
    engine = create_engine(DATABASE_URL)
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("Sucesso na conexao com PostgreSQL.")
except SQLAlchemyError as e:
    print("Erro ao conectar ao banco:", e)
    raise SystemExit

In [ ]:
caminho_csv = "../Data/df_Social_Media_Tratado.csv"

In [ ]:
try:
    for i, chunk in enumerate(pd.read_csv(caminho_csv, chunksize=1000)):
        chunk.to_sql(
            name='campanhas',
            con=engine,
            if_exists='append',  # 'replace' na primeira execução para recriar
            index=False,
            chunksize=20000
        )
        print(f"Lote {i+1} inserido ({len(chunk)} linhas).")

    print("Sucesso na inserção!!")
except SQLAlchemyError as exp_erro:
    print("Erro na exportação para o banco:", exp_erro)
except FileNotFoundError:
    print("Arquivo CSV não encontrado:", caminho_csv)
except Exception as erro:
    print("Erro:", erro)